In [ ]:
#Copyright (c) Microsoft Corporation. All rights reserved.
#Licensed under the MIT License.

In [ ]:
#Object Detection transfer learning training using tensorflow mobilenetssdlitev2 model on Azure ML
#Warning copying *.pb, *.bin, or, *.blob using the web interface can corrupt the files. If needed download and use Azure storage explorer or the CLI 

In [ ]:
#Setup workspace
import azureml.core
from azureml.core import Workspace
print(azureml.core.VERSION)

In [ ]:
#Get current directory
modelroot = !pwd

In [ ]:
#save current directory for reference
modelroot = modelroot[0]
modelroot

In [ ]:
#install Tensor flow models and scripts, v1.13.0 needed for OpenVINO
repository = '--depth 1 --branch v1.13.0 https://github.com/tensorflow/models.git'
!pip install tf-slim
!git clone $repository

In [ ]:
#install required TF packages
!sudo -s apt-get install -qq protobuf-compiler python-tk

In [ ]:
#install required TF packages
!pip install Cython contextlib2 pillow lxml matplotlib PyDrive pycocotools build utils dataclasses install azure-iot-device azure-iot-hub

In [ ]:
#setup python path for TF
import os
import sys
cwd = os.getcwd()
cwd
sys.path.append(cwd)
research = cwd + '/models/research'
sys.path.append(research)
slim = cwd + '/models/research/slim'
sys.path.append(slim)
sys.path
%set_env PYTHONPATH=''

os.environ['PYTHONPATH'] = research + ":" + slim +  ":" + cwd

os.environ['PYTHONPATH']

In [ ]:
!pwd

In [ ]:
#Update protobuf for TF
!curl -OL 'https://github.com/google/protobuf/releases/download/v3.2.0/protoc-3.2.0-linux-x86_64.zip'

In [ ]:
#Update protobuf for TF
unzipCMD = '-o protoc-3.2.0-linux-x86_64.zip -d protoc3 '
!unzip $unzipCMD

In [ ]:
#Update protobuf for TF
moveCMD = 'mv protoc3/bin/* /usr/local/bin/' 
!sudo $moveCMD

In [ ]:
#Update protobuf for TF
moveCMD = 'mv protoc3/include/* /usr/local/include/'
!sudo $moveCMD

In [ ]:
!pwd

In [ ]:
researchfolder =  modelroot + '/models/research'
%cd $researchfolder

protocCMD = '/usr/local/bin/protoc ' + 'object_detection/protos/*.proto --python_out=.'
!$protocCMD

In [ ]:
#install tensorflow v1.15 required for OpenVINO model conversion
!pip install tensorflow-gpu==1.15

In [ ]:
#check Tensorflow version
import tensorflow.compat.v1 as tf
#import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
#cycle jupyter kernel if needed to pickup tensorflow change from 2.1 to 1.15.0

In [ ]:
import tensorflow.compat.v1 as tf
#import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
#Run setup for TF models build
args = researchfolder + '/setup.py build'
!python $args

In [ ]:
#Run setup for TF models install
args = researchfolder + '/setup.py install'
!python $args

In [ ]:
#run tensor flow model builder test
args = modelroot + '/models/research/object_detection/builders/model_builder_test.py'
!python $args

In [ ]:
%cd $modelroot

In [ ]:
#Create data folder for training dataset and model
!mkdir data

In [ ]:
%cd data

In [ ]:
#download training data set
!curl -OL 'https://github.com/RomRoc/objdet_train_tensorflow_colab/raw/master/dog_dataset.zip'

In [ ]:
#unzip and organize dataset
from zipfile import *
fileName = 'dog_dataset.zip'
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

image_files=os.listdir('images')
im_files=[x.split('.')[0] for x in image_files]
with open('annotations/trainval.txt', 'w') as text_file:
  for row in im_files:
    text_file.write(row + '\n')

In [ ]:
#unzip and organize dataset
%cd ./annotations
!mkdir trimaps

In [ ]:
#unzip and organize dataset
from PIL import Image
image = Image.new('RGB', (640, 480))

for filename in os.listdir('xmls'):
  filename = os.path.splitext(filename)[0]
  image.save('trimaps/' + filename + '.png')

In [ ]:
#create category labels file for Tensorflow training and validation files (label map)
labelMapFileName = modelroot + '/models/research/object_detection/data/Dog.pbtxt'
print(labelMapFileName)

In [ ]:
%%writefile $labelMapFileName
item {
  id: 1
  name: 'dog'
}

In [ ]:
#fix to remove ^M nonprintable char from end of string lines in file created above
#to see issue run "!cat -v $labelMapFileName" before and after fix
with open(labelMapFileName,'r') as file:
    labelMapFile = file.read()
updateFile = open(labelMapFileName,"w")
updateFile.writelines(labelMapFile)
updateFile.close() 

In [ ]:
#create Tensorflow training data record files
os.chdir(modelroot + '/data')
args = modelroot + '/models/research/object_detection/dataset_tools/create_pet_tf_record.py --label_map_path=' + modelroot + '/models/research/object_detection/data/Dog.pbtxt --data_dir=./ --output_dir=./ --num_shards=1'

!python $args

In [ ]:
#update training record name
!mv pet_faces_train.record-00000-of-00001 tf_train.record

In [ ]:
#update validation record name
!mv pet_faces_val.record-00000-of-00001 tf_val.record

In [ ]:
#download pretrained model: SSD Lite MobileNet V2 COCO
#For doing transfer learning with SSD Mobilenet V2 COCO just uncomment the script
#https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md
!curl -OL 'http://download.tensorflow.org/models/object_detection/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz'
#!curl -OL 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v2_coco_2018_03_29.tar.gz'

In [ ]:
MODEL_FILE = modelroot + '/data/ssdlite_mobilenet_v2_coco_2018_05_09.tar.gz'
#MODEL_FILE = modelroot + '/data/ssd_mobilenet_v2_coco_2018_03_29.tar.gz'

In [ ]:
#unzip model
import os
import shutil
import glob
import urllib
import tarfile
import urllib.request

s=MODEL_FILE
tar = tarfile.open(s)
tar.extractall()
tar.close()

In [ ]:
#prepare model config file paths and tensorflow configuration file for SSDLiteV2 Model
configFileName = modelroot + '/models/research/object_detection/samples/configs/ssdlite_mobilenet_retrained.config'
fine_tune_checkpoint = '"' + modelroot + '/data/ssdlite_mobilenet_v2_coco_2018_05_09/model.ckpt' + '"'
input_path_train = '"' + modelroot + '/data/tf_train.record' + '"'
label_map_path = '"' + modelroot + '/models/research/object_detection/data/Dog.pbtxt' + '"'
input_path_eval = '"' + modelroot + '/data/tf_val.record' + '"'

In [ ]:
'''
#prepare model config file paths and tensorflow configuration file for SSDV2 Model
configFileName = modelroot + '/models/research/object_detection/samples/configs/ssd_mobilenet_retrained.config'
fine_tune_checkpoint = '"' + modelroot + '/data/ssd_mobilenet_v2_coco_2018_05_09/model.ckpt' + '"'
input_path_train = '"' + modelroot + '/data/tf_train.record' + '"'
label_map_path = '"' + modelroot + '/models/research/object_detection/data/Dog.pbtxt' + '"'
input_path_eval = '"' + modelroot + '/data/tf_val.record' + '"'
'''

In [ ]:
%%writefile $configFileName
model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.99999989895e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.0299999993294
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.999700009823
          center: true
          scale: true
          epsilon: 0.0010000000475
          train: true
        }
      }
      use_depthwise: true
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    box_predictor {
      convolutional_box_predictor {
        conv_hyperparams {
          regularizer {
            l2_regularizer {
              weight: 3.99999989895e-05
            }
          }
          initializer {
            truncated_normal_initializer {
              mean: 0.0
              stddev: 0.0299999993294
            }
          }
          activation: RELU_6
          batch_norm {
            decay: 0.999700009823
            center: true
            scale: true
            epsilon: 0.0010000000475
            train: true
          }
        }
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        use_dropout: false
        dropout_keep_probability: 0.800000011921
        kernel_size: 3
        box_code_size: 4
        apply_sigmoid_to_scores: false
        use_depthwise: true
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.20000000298
        max_scale: 0.949999988079
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.333299994469
      }
    }
    post_processing {
      batch_non_max_suppression {
        score_threshold: 0.300000011921
        iou_threshold: 0.600000023842
        max_detections_per_class: 100
        max_total_detections: 100
      }
      score_converter: SIGMOID
    }
    normalize_loss_by_num_matches: true
    loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000
        iou_threshold: 0.990000009537
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
  }
}
train_config {
  batch_size: 1
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
  optimizer {
    rms_prop_optimizer {
      learning_rate {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.00400000018999
          decay_steps: 800720
          decay_factor: 0.949999988079
        }
      }
      momentum_optimizer_value: 0.899999976158
      decay: 0.899999976158
      epsilon: 1.0
    }
  }
  fine_tune_checkpoint: $fine_tune_checkpoint
  num_steps: 10
  fine_tune_checkpoint_type: "detection"
}
train_input_reader {
  label_map_path: $label_map_path
  tf_record_input_reader {
    input_path: $input_path_train
  }
}
eval_config {
  num_examples: 30000
  max_evals: 30000
  use_moving_averages: false
}
eval_input_reader {
  label_map_path: $label_map_path
  shuffle: false
  num_readers: 1
  tf_record_input_reader {
    input_path: $input_path_eval
  }
}

In [ ]:
#update config file paths
with open(configFileName,'r') as file:
    configFile = file.read()
configFile = configFile.replace('$fine_tune_checkpoint', fine_tune_checkpoint)
configFile = configFile.replace('$label_map_path', label_map_path)
configFile = configFile.replace('$input_path_train', input_path_train)
configFile = configFile.replace('$input_path_eval', input_path_eval)
updateFile = open(configFileName,"w")
updateFile.writelines(configFile)
updateFile.close() 


In [ ]:
import tensorflow.compat.v1 as tf
#import tensorflow as tf
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
model_dir =  modelroot + '/data/retrained'

In [ ]:
#Run tensor flow fine tuning training, to train on CPU instead of GPU uncomment the 'CUDA_VISIBLE_DEVICES' line below
#If training for more than 10,000 iterations the notebook will stop updating output after a while, but the training is still happening in the background
#In that case run !nvidia-smi to check the current load on the GPU to see if training is still happening
#You can watch for new ckeckpoint files being written about every 10 minutes with the command !ls $modelroot/data/retrained

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
!python $modelroot/models/research/object_detection/model_main.py --pipeline_config_path=$configFileName --model_dir=$model_dir --alsologtostderr --num_train_steps=30000 --num_eval_steps=30000

In [ ]:
#free up GPU after run
from numba import cuda
cuda.select_device(1)
cuda.close()

In [ ]:
%cd $modelroot/data

In [ ]:
#get the last checkpoint to use for exporting a frozen graph
import re
lst = os.listdir('retrained')
lf = filter(lambda k: 'model.ckpt-' in k, lst)
fileList = str(list(lf))
checkPointNumbers = re.findall(r'[0-9]+', fileList)
checkPointNumbers = [int(i) for i in checkPointNumbers]  
last_model = 'model.ckpt-' + str(max(checkPointNumbers))
print(last_model)

In [ ]:
#export frozen graph
!python $modelroot/models/research/object_detection/export_inference_graph.py --input_type=image_tensor --pipeline_config_path=$configFileName --output_directory=fine_tuned_model --trained_checkpoint_prefix=retrained/$last_model

In [ ]:
#test inference on photo using frozen graph

import tensorflow.compat.v1 as tf
import numpy as np
import os
import cv2
#import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from matplotlib import pyplot as plt
from PIL import Image

# path to the frozen graph:
PATH_TO_FROZEN_GRAPH = modelroot + '/data/fine_tuned_model/frozen_inference_graph.pb'
#PATH_TO_FROZEN_GRAPH = modelroot + '/data/ssdlite_mobilenet_v2_coco_2018_05_09/frozen_inference_graph.pb'

# path to the label map
PATH_TO_LABEL_MAP = modelroot + '/models/research/object_detection/data/Dog.pbtxt'
#PATH_TO_LABEL_MAP = modelroot + '/models/research/object_detection/data/mscoco_label_map.pbtxt'


# number of classes 
NUM_CLASSES = 1

#cap = cv2.VideoCapture(0)

#reads the frozen graph
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

label_map = label_map_util.load_labelmap(PATH_TO_LABEL_MAP)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Detection
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        
          # Read frame from camera
          #ret, image_np = cap.read()

          #image_np = cv2.imread(modelroot + '/data/images/blank_wall.jpg')  
          #image_np = cv2.imread(modelroot + '/data/images/nodog_001.jpg')  
          image_np = cv2.imread(modelroot + '/data/images/dog_047.jpg')


          # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
          image_np_expanded = np.expand_dims(image_np, axis=0)
          # Extract image tensor
          image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
          # Extract detection boxes
          boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
          # Extract detection scores
          scores = detection_graph.get_tensor_by_name('detection_scores:0')
          # Extract detection classes
          classes = detection_graph.get_tensor_by_name('detection_classes:0')
          # Extract number of detections
          num_detections = detection_graph.get_tensor_by_name('num_detections:0')
          # Actual detection.
          (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          print ( boxes,scores,classes,num_detections)
          # Visualization of the results of a detection.
          vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordin50es=True,
              line_thickness=3,
              min_score_thresh=.85,
              max_boxes_to_draw=100
              )

In [ ]:
# display test inference photo

from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image

%matplotlib inline
plt.figure(figsize = (12,8))
imshow(image_np)

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
interactive_auth = InteractiveLoginAuthentication(tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")
ws = Workspace(subscription_id="SUBSCRIPTIONID",
               resource_group="RESOURCEGROUPNAME",
               workspace_name="MLWORKSPACE",
               auth=interactive_auth)
ws.get_details()
print (ws)

In [ ]:
#reload workspace to register model

from azureml.core import Workspace
ws.get_details()

In [ ]:
trained_model_path = modelroot + '/data/fine_tuned_model/'
print(trained_model_path)

In [ ]:
#Register the trained model. Once register you'll find the model in the Models section on the left pane
from azureml.core.model import Model

model = Model.register(model_path = trained_model_path,
                      model_name = "dog_ssdv2lite",
                      tags = {"data": "ssd_mobilenetv2lite", "model": "object_detection", "type": "ssd_mobilenetv2lite"},
                      description = "Retrained dog detection based on ssd_mobilenetv2lite",
                      workspace = ws)

In [ ]:
frozen_inference_graph = modelroot + '/data/fine_tuned_model/frozen_inference_graph.pb'


In [ ]:
#Install Intel® Distribution of OpenVINO™ toolkit for Linux* Using APT Repository
#https://docs.openvinotoolkit.org/2019_R2/_docs_install_guides_installing_openvino_apt.html

!curl -OL -o GPG-PUB-KEY-INTEL-OPENVINO-2020 'https://apt.repos.intel.com/openvino/2020/GPG-PUB-KEY-INTEL-OPENVINO-2020'

In [ ]:
#install OpenVINO
aptKey = 'GPG-PUB-KEY-INTEL-OPENVINO-2020'
!sudo apt-key add $aptKey

In [ ]:
#install OpenVINO
!sudo apt-key list

In [ ]:
#install OpenVINO
echo_args = '"deb https://apt.repos.intel.com/openvino/2020 all main" | sudo tee /etc/apt/sources.list.d/intel-openvino-2020.list'
!echo $echo_args

In [ ]:
#install OpenVINO
!sudo apt update

In [ ]:
#install OpenVINO
!apt-cache search openvino

In [ ]:
#install OpenVINO
!sudo apt-cache search intel-openvino-dev-ubuntu16

In [ ]:
#install OpenVINO
package = 'intel-openvino-dev-ubuntu16-2020.2.130'
!sudo apt install -y $package

In [ ]:
modelroot = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/dw-gpu/code/users/<USERNAME>'
%cd $modelroot/data

In [ ]:
#Use OpenVINO to converto frozen graph PB to IR format

#import OpenVINO env vars
convertCMD = 'source /opt/intel/openvino/bin/setupvars.sh && '

#convert frozen graph to Intermediate Representation
convertCMD += '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py '
convertCMD += '--input_model ' + modelroot + '/data/fine_tuned_model/frozen_inference_graph.pb '
convertCMD += '--tensorflow_object_detection_api_pipeline_config ' + modelroot + '/data/fine_tuned_model/pipeline.config '
convertCMD += '--tensorflow_use_custom_operations_config /opt/intel/openvino/deployment_tools/model_optimizer/extensions/front/tf/ssd_v2_support.json '
convertCMD += '--reverse_input_channels'

print(convertCMD)
!$convertCMD$convertCMD


In [ ]:
#Use OpenVINO to compile IR format to blob

#import OpenVINO env vars
convertCMD = 'source /opt/intel/openvino/bin/setupvars.sh && '

#compile IR to blob 
convertCMD += '/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/myriad_compile '
convertCMD += '-m frozen_inference_graph.xml '
convertCMD += '-o ssdlite_mobilenet_v2.blob '
convertCMD += '-VPU_MYRIAD_PLATFORM VPU_MYRIAD_2480 '
convertCMD += '-VPU_NUMBER_OF_SHAVES 8 '
convertCMD += '-VPU_NUMBER_OF_CMX_SLICES 8 '
convertCMD += '-ip U8 '
convertCMD += '-op FP32'

print(convertCMD)
!$convertCMD

In [ ]:
#package up blob for delevery to devkit
#clean recreate directory if it exists and create it if not 
cmd = 'rm -rf blob && mkdir -p blob'
!$cmd

In [ ]:
#package up blob for delevery to devkit
!cp ssdlite_mobilenet_v2.blob blob/ssdlite_mobilenet_v2.blob

In [ ]:
%cd blob

In [ ]:
#create labels file for devkit
# do not remove zeroindex, it is needed to align graph classification index with lables index
labelFileName = 'labels.txt'

In [ ]:
%%writefile $labelFileName
zeroindex
dog

In [ ]:
#fix to remove ^M nonprintable char from end of string lines in file created above
#to see issue run "!cat -v $labelFileName" before and after fix
with open(labelFileName,'r') as file:
    labelFile = file.read()
updateFile = open(labelFileName,"w")
updateFile.writelines(labelFile)
updateFile.close() 

In [ ]:
#create json file for devkit
jsonConfigFileName = 'config.json'

In [ ]:
%%writefile $jsonConfigFileName
{
    "DomainType": "ssd100",
    "LabelFileName": "labels.txt",
    "ModelFileName": "ssdlite_mobilenet_v2.blob"
}

In [ ]:
'''For SSDV2 Model
%%writefile $jsonConfigFileName
{
    "DomainType": "ssd100",
    "LabelFileName": "labels.txt",
    "ModelFileName": "ssd_mobilenet_v2.blob"
}
'''

In [ ]:
#fix to remove ^M nonprintable char from end of string lines in file created above
#to see issue run "!cat -v $jsonConfigFileName" before and after fix
with open(jsonConfigFileName,'r') as file:
    jsonConfigFile = file.read()
updateFile = open(jsonConfigFileName,"w")
updateFile.writelines(jsonConfigFile)
updateFile.close() 

In [ ]:
#package up model and support files for dev kit
zipCMD = 'zip -r model.zip *'
!$zipCMD

In [ ]:
#Reload workspace details for module twin update

from azureml.core import Workspace
ws.get_details()

In [ ]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

In [ ]:
#set data path for model.zip and upload
data_path = 'model'
ds.upload(src_dir='.', target_path=data_path, overwrite=True)

In [ ]:
!pip install azure-storage-blob==2.1.0
!pip install msrest

In [ ]:
#Use one of the options to generate Saas Url

In [ ]:
#Option1 - Generate download SAS URL for model.zip
from datetime import datetime, timedelta
from azure.storage.blob import (
    BlockBlobService,
    ContainerPermissions,
    BlobPermissions,
    PublicAccess,
)

AZURE_ACC_NAME = ds.account_name
AZURE_PRIMARY_KEY = ds.account_key
AZURE_CONTAINER = ds.container_name
AZURE_BLOB=ds.name
AZURE_File=data_path+'/model.zip'

block_blob_service = BlockBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)
sas_url = block_blob_service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=360))
downloadurl ='https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url
print('https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url)
print(sas_url)

In [ ]:
#Option2 - Generate download SAS URL for model.zip
from azure.storage.blob.baseblobservice import BaseBlobService,ContainerPermissions,BlobPermissions
from datetime import datetime, timedelta
AZURE_ACC_NAME = ds.account_name
AZURE_PRIMARY_KEY = ds.account_key
AZURE_CONTAINER = ds.container_name
AZURE_BLOB=ds.name
AZURE_File=data_path+'/model.zip'
service = BaseBlobService(account_name=AZURE_ACC_NAME, account_key=AZURE_PRIMARY_KEY)
sas_url  = service.generate_blob_shared_access_signature(AZURE_CONTAINER,AZURE_File,permission=BlobPermissions.READ,expiry= datetime.utcnow() + timedelta(hours=48))
downloadurl ='https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url
print('https://'+AZURE_ACC_NAME+'.blob.core.windows.net/'+AZURE_CONTAINER+'/'+AZURE_File+'?'+sas_url)
print(sas_url)

In [ ]:
#Perform Module twin update
#Incorporate the connection string, device_id and the module_id values from your IoTHub

#!pip install azure-iot-hubprint
print(downloadurl)
import sys
from azure.iot.hub import IoTHubRegistryManager
from azure.iot.hub.models import Twin, TwinProperties

#Incorporate Iothub connection string and the default module name
#Go to Https://portal.azure.com
#Select your IoTHub
#Click on Shared access policies
#click service on right
#Copy the iothub connection string primary key

CONNECTION_STRING = 'HostName=<IOTHUBCONNECTIONSTRING>'
DEVICE_ID = '<DEVICENAME>'
MODULE_ID = "azureeyemodule"

try:
    # RegistryManager
    iothub_registry_manager = IoTHubRegistryManager(CONNECTION_STRING)

    module_twin = iothub_registry_manager.get_module_twin(DEVICE_ID, MODULE_ID)
    print ( "" )
    print ( "Module twin properties before update    :" )
    print ( "{0}".format(module_twin.properties) )

    # Update twin
    twin_patch = Twin()
    twin_patch.properties = TwinProperties(desired={"ModelZipUrl": downloadurl})
    updated_module_twin = iothub_registry_manager.update_module_twin(
        DEVICE_ID, MODULE_ID, twin_patch, module_twin.etag
    )
    print ( "" )
    print ( "Module twin properties after update     :" )
    print ( "{0}".format(updated_module_twin.properties) )

except Exception as ex:
    print ( "Unexpected error {0}".format(ex) )
except KeyboardInterrupt:
    print ( "IoTHubRegistryManager sample stopped" )

In [ ]:
# The trained model will get pushed to the IoT Edge device via module twin update method
# Check model inferencing by connecting monitor to the devkit or by installing VLC media player : 
#Install VLC from https://www.videolan.org/vlc/ and install on “Windows” to check the camera function of “Azure Eye”.

#Check video stream:
#1.	Select Media -> Open Network Stream…
#2.	Input the network stream: “rtsp://[ip of PE-101]:8554/result” then click “Play” button.
#3. or use webstream http://<ipaddressofcamera>:3000

In [ ]:
# delete cpu compute
"""
mycompute = AmlCompute(workspace=ws, name='dw-cpu1')
mycompute.delete()

# delete gpu compute
mycompute = AmlCompute(workspace=ws, name='dw-gpu')
mycompute.delete()
"""

In [ ]:
# delete workspace
#ws.delete(delete_dependent_resources=True)
